# Salient Predictions Ski-Cast

At the start of the Northern Hemisphere ski season, Salient predicts snow accumulation at 90 IKON and Epic resorts.

In [ ]:
import datetime
import hashlib
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

# Prevent wrapping on tables for readability
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

sk.set_file_destination("ski_example")
sk.login("SALIENT_USERNAME", "SALIENT_PASSWORD")

## Customize the calculation


In [ ]:
# Timeframe for analysis:
year = 2025
norm_years = 15  # number of historical years to analyze
start_month = 10  # first month of the northern hemisphere snow accumulation season
end_month = 4  # final month of the ski season
today = pd.to_datetime(datetime.date.today())

# Request a Monday to get 126 days of leads:
fcst_date = pd.to_datetime(f"{year}-12-01")
hist_start = pd.to_datetime(f"{year-norm_years-1}-{start_month}-01")
fcst_end = pd.to_datetime(f"{year+1}-{end_month}-01") - pd.Timedelta(days=1)

force = False
verbose = False
debias = False

vars = ["temp", "precip"]
freq = "daily"

### Define all of the IKON/EPIC resorts worldwide

We'll define all the locations of interest in code here to keep the example self-contained. Often you'll list your locations in a separate CSV file


In [ ]:
geo = pd.DataFrame(
    [  # lon      lat     name      region    pass
        (137.861, 36.690, "Hakuba", "japan", "epic"),
        (140.874, 42.739, "Rusutsu", "japan", "epic"),
        (140.688, 42.864, "Niseko", "japan", "ikon"),
        (138.175, 37.027, "Lotte Arai", "japan", "ikon"),
        # Europe --------------------------------
        (6.8632749, 45.924065, "Chamonix", "europe", "ikon"),
        (7.7522747, 46.0222204, "Zermatt", "europe", "ikon"),
        (8.5916293, 46.6324621, "Andermatt-Sedrun", "europe", "epic"),
        (12.3925407, 47.4492375, "Kitzbuhel", "europe", "ikon"),
        (1.4707674, 42.5729217, "Arinsal", "europe", "ikon"),
        (1.499825, 42.6317345, "Ordino Arcalís", "europe", "ikon"),
        (1.6462281, 42.5783833, "Grandvalira", "europe", "ikon"),
        (11.6520936, 46.5739752, "Dolomiti", "europe", "ikon"),
        # Pacific Northwest -------------------
        (-123.204545, 49.396018, "Cypress", "pnw", "ikon"),
        (-121.6781891, 44.0024701, "Bachelor", "pnw", "ikon"),
        (-121.0890197, 47.7448119, "Stevens Pass", "pnw", "epic"),
        (-122.9486474, 50.1149639, "Whistler", "pnw", "epic"),
        (-121.4747533, 46.9352963, "Crystal Mtn", "pnw", "ikon"),
        (-121.4257485, 47.4442426, "Alpental", "pnw", "ikon"),
        (-121.4164161, 47.4245711, "Snoqualmie", "pnw", "ikon"),
        # Rocky Mountains (USA) ---------------------------
        (-106.818227, 39.1862601, "Aspen Mtn", "rockies", "ikon"),
        (-106.8045169, 40.4571991, "Steamboat", "rockies", "ikon"),
        (-106.3549717, 39.6061444, "Vail", "rockies", "epic"),
        (-106.1516265, 39.501419, "Copper", "rockies", "ikon"),
        (-106.0676088, 39.4808351, "Breckenridge", "rockies", "epic"),
        (-105.9437656, 39.6075962, "Keystone", "rockies", "epic"),
        (-105.8719397, 39.6425118, "A-Basin", "rockies", "ikon"),
        (-105.762488, 39.8868392, "Winter Park", "rockies", "ikon"),
        (-105.5826786, 39.9372203, "Eldora", "rockies", "ikon"),
        (-105.4545, 36.5959999, "Taos", "rockies", "ikon"),
        # (-106.9490961, 39.2083984, "Aspen Snowmass","rockies","ikon"), # redundant Ajax
        # (-106.8610687, 39.2058029, "Buttermilk","rockies","ikon"),  # redundant Ajax
        # (-106.8553613, 39.1824124, "Aspen Highlands","rockies","ikon"),  # redundant Ajax
        # (-106.5167109, 39.6042863, "Beaver Creek","rockies","epic"), # redundant Vail
        # Canadian Interior ---------------------------
        (-118.1630779, 50.9583858, "Revelstoke", "ca_west", "ikon"),
        (-117.8194705, 49.1024147, "RED", "ca_west", "ikon"),
        (-116.2380671, 50.4602801, "Panorama", "ca_west", "ikon"),
        (-116.1621717, 51.4419206, "Lake Louise", "ca_west", "ikon"),
        (-115.7840699, 51.0780997, "Banff", "ca_west", "ikon"),
        (-115.5982699, 51.2037624, "Norquay", "ca_west", "ikon"),
        (-115.5707632, 51.1751675, "SkiBig3", "ca_west", "ikon"),
        # Northeast USA & Canada ---------------
        (-72.9278443, 43.0906207, "Stratton", "na_east", "ikon"),
        (-72.9204014, 42.9602444, "Mt Snow", "na_east", "epic"),
        (-72.8944139, 44.1359019, "Sugarbush", "na_east", "ikon"),
        (-72.7967531, 43.6262922, "Killington", "na_east", "ikon"),
        (-72.7814124, 44.5303066, "Stowe", "na_east", "epic"),
        (-72.7170416, 43.4018257, "Okemo", "na_east", "epic"),
        (-72.08014, 43.331889, "Sunapee", "na_east", "epic"),
        (-71.8655176, 43.0198715, "Crotched", "na_east", "epic"),
        (-71.6336041, 44.0563456, "Loon", "na_east", "ikon"),
        (-71.2393036, 44.2640724, "Wildcat", "na_east", "epic"),
        (-71.229443, 44.082771, "Attitash", "na_east", "epic"),
        (-70.8568727, 44.4734182, "Sunday River", "na_east", "ikon"),
        (-70.3085109, 45.0541811, "Sugarloaf", "na_east", "ikon"),
        (-72.5257, 44.9242, "Jay Peak", "na_east", "indie"),
        (-83.6777778, 40.3180556, "Mad River", "na_east", "epic"),
        (-74.5852526, 46.2096417, "Tremblant", "na_east", "ikon"),
        (-74.2567116, 42.2937298, "Windham", "na_east", "ikon"),  # NY Catskills
        (-74.2246402, 42.2028811, "Hunter", "na_east", "epic"),  # NY Catskills
        # (-72.842512, 43.6621499, "Pico","na_east","ikon"), # redundant Killington
        # USA West (Sierras & Wasatch) ---------------
        (-120.2483913, 39.1906091, "Palisades Tahoe", "na_west", "ikon"),
        (-120.1210934, 39.2745678, "Northstar", "na_west", "epic"),
        (-120.0651665, 38.6847514, "Kirkwood", "na_west", "epic"),
        (-119.9428424, 38.9569241, "Heavenly", "na_west", "epic"),
        (-119.8859331, 50.8844311, "Sun Peaks", "na_west", "ikon"),
        (-119.0906293, 37.7679169, "June", "na_west", "ikon"),
        (-119.0267806, 37.6510972, "Mammoth", "na_west", "ikon"),
        (-117.036177, 34.2248821, "Snow Valley", "na_west", "ikon"),
        (-116.8892717, 34.2364081, "Snow Summit", "na_west", "ikon"),
        (-116.8608572, 34.2276766, "Bear Mtn", "na_west", "ikon"),
        (-116.6227441, 48.3679757, "Schweitzer", "na_west", "ikon"),
        (-114.3542874, 43.6949128, "Sun Valley", "na_west", "ikon"),
        (-114.3461537, 43.6820566, "Dollar Mtn", "na_west", "ikon"),
        (-111.8571529, 41.2161404, "Snowbasin", "na_west", "ikon"),
        (-111.6385807, 40.5884218, "Alta", "na_west", "ikon"),
        (-111.5079947, 40.6514199, "Park City", "na_west", "epic"),
        (-111.4012076, 45.2857289, "Big Sky", "na_west", "ikon"),
        (-110.8279183, 43.5875453, "Jackson Hole", "na_west", "ikon"),
        (-106.9878231, 38.8697146, "Crested Butte", "na_west", "ikon"),
        # (-111.6563885, 40.5810814, "Snowbird","na_west"), # redundant Alta
        # (-111.591885, 40.619852, "Solitude","na_west"), # redundant Alta
        # (-111.583187, 40.598019, "Brighton","na_west"), # redundant Alta
        # (-111.478306, 40.63738, "Deer Valley", "na_west", "ikon"), # redundant Park City
        # Middle North America -----------------------------
        (-94.9707416, 39.4673048, "Snow Creek", "na_mid", "epic"),  # Kansas City
        (-92.7878062, 44.8576608, "Afton", "na_mid", "epic"),  # Minneapolis
        (-90.6506898, 38.5353168, "Hidden Valley", "na_mid", "epic"),
        (-88.1876602, 42.4989548, "Wilmot", "na_mid", "epic"),
        (-86.5122305, 38.5555868, "Paoli", "na_mid", "epic"),
        (-84.930067, 45.162884, "Boyne", "na_mid", "ikon"),
        (-83.8115217, 42.54083, "Mt. Brighton", "na_mid", "epic"),
        (-81.5632108, 41.2640987, "Boston Mills", "na_mid", "epic"),
        (-81.259745, 41.52687, "Alpine Valley", "na_mid", "epic"),
        (-80.3122216, 44.5037818, "Blue Mtn", "na_mid", "ikon"),
        (-79.9960444, 38.4118566, "Snowshoe", "na_mid", "ikon"),
        (-79.2977032, 40.0229768, "7 Springs", "na_mid", "epic"),
        (-79.2581204, 40.058031, "Hidden Valley 2", "na_mid", "epic"),
        (-79.1657908, 40.1638728, "Laurel", "na_mid", "epic"),
        (-77.9333126, 39.7417652, "Whitetail", "na_mid", "epic"),
        (-77.375459, 39.76366, "Liberty", "na_mid", "epic"),
        (-76.9275492, 40.1094506, "Roundtop", "na_mid", "epic"),
        (-75.6563315, 41.1091686, "Jack Frost", "na_mid", "epic"),
        (-75.601282, 41.050189, "Big Boulder", "na_mid", "epic"),
        # (-84.926535, 45.4647239, "Boyne Highlands", "na_east", "ikon"), # redundant Boyne
    ],
    columns=["lon", "lat", "name", "region", "pass"],
)
assert not geo.duplicated(
    subset=["lon", "lat"]
).any(), f"Duplicate coordinates found: {geo[geo.duplicated(subset=['lon', 'lat'], keep=False)].sort_values(['lon', 'lat'])}"

# We define a "region" for each ski resort to group them for plotting purposes,
# using Tol's colorblind-friendly "vibrant" palette.
# https://cran.r-project.org/web/packages/khroma/vignettes/tol.html
geo_colors = {
    "japan": "#004488",  # blue
    "pnw": "#009988",  # teal
    "rockies": "#CC3311",  # red
    "ca_west": "#EE3377",  # magenta
    "na_east": "#DDAA33",  # yellow
    "europe": "#33BBEE",  # cyan
    "na_west": "#EE7733",  # orange
    "na_mid": "#777777",  # light grey
}
geo["color"] = geo["region"].map(geo_colors)

locs = sk.Location(
    location_file=sk.upload_location_file(
        lats=geo.lat,
        lons=geo.lon,
        names=geo.name,
        geoname="resorts",
        force=force,
        debias=None,
        region=geo.region,
        color=geo.color,
    )
)

print(locs)
print(geo)

location file: resorts.csv
           lon        lat         name  region  pass    color
0   137.861000  36.690000       Hakuba   japan  epic  #004488
1   140.874000  42.739000      Rusutsu   japan  epic  #004488
2   140.688000  42.864000       Niseko   japan  ikon  #004488
3   138.175000  37.027000   Lotte Arai   japan  ikon  #004488
4     6.863275  45.924065     Chamonix  europe  ikon  #33BBEE
..         ...        ...          ...     ...   ...      ...
87  -77.933313  39.741765    Whitetail  na_mid  epic  #777777
88  -77.375459  39.763660      Liberty  na_mid  epic  #777777
89  -76.927549  40.109451     Roundtop  na_mid  epic  #777777
90  -75.656331  41.109169   Jack Frost  na_mid  epic  #777777
91  -75.601282  41.050189  Big Boulder  na_mid  epic  #777777

[92 rows x 6 columns]


## Cluster

Our locations span multiple continents, and the `forecast_timeseries` endpoint requires each `location_file` to contain only points from a single continent.  Let's beak the comprehensive list into regional clusters that don't span continent boundaries.

In [ ]:
loc = locs.cluster(cluster_size=256, upload="changed", verbose=verbose)
force = force or loc.any_changed

print(loc)

location file: ['resorts_6_00.csv', 'resorts_3_00.csv', 'resorts_1_00.csv']


## Acquire the data

For each of the ski resorts, we will get the daily forecast of temperature and precipitation as of the beginning of the season. Then we will also get the historical observed conditions, calculate snowfall, and merge them into a single dataset for later analysis.


In [ ]:
fcst_files = sk.forecast_timeseries(
    loc=loc,
    variable=vars,
    field="vals_ens",
    model="gem",
    timescale=freq,
    date=fcst_date,
    force=force,
    debias=debias,
    verbose=verbose,
    strict=False,
)
print(fcst_files)

                                           file_name     location_file variable
0  ski_example/forecast_timeseries_24ceb810ecd865...  resorts_6_00.csv     temp
1  ski_example/forecast_timeseries_fd48b5c01d0261...  resorts_6_00.csv   precip
2  ski_example/forecast_timeseries_94c21af12c87e2...  resorts_3_00.csv     temp
3  ski_example/forecast_timeseries_0a88fdf3964280...  resorts_3_00.csv   precip
4  ski_example/forecast_timeseries_d43e51be58ffe2...  resorts_1_00.csv     temp
5  ski_example/forecast_timeseries_4a9ee4cb4b0999...  resorts_1_00.csv   precip


In [ ]:
# If there was an error when downloading, remove the failing files:
na_count = fcst_files["file_name"].isna().sum()
if na_count > 0:
    print(f"Removing {na_count} rows with NA file_name:")
    print(fcst_files[fcst_files["file_name"].isna()])

    # Filter out NA file names
    fcst_files = fcst_files[fcst_files["file_name"].notna()]
    print(fcst_files)

Because we are requesting multiple location_files and `variable`s, `fcst_files` is a
table with multiple forecasts. Let's combine all of them, and bring in the extra meta-information like `color` from the original `location_file` csvs using the `sk.merge_location_data` function.

In [ ]:
def preprocess_location_metadata(ds):
    """Merge additional columns from location_file.csv into the xarray datasets."""
    source = ds.encoding["source"]
    source_base = os.path.basename(source)
    idx = fcst_files.index[fcst_files["file_name"].apply(os.path.basename) == source_base][0]
    ds = ds.rename({"vals_ens": fcst_files.variable[idx]})
    idx_file = os.path.join(sk.get_file_destination(), fcst_files.loc[idx, "location_file"])
    return sk.merge_location_data(ds, idx_file, as_data_vars=False)


# Group files by their location_file and process each group
datasets = [
    xr.open_mfdataset(
        group["file_name"].values,
        combine="by_coords",
        preprocess=preprocess_location_metadata,
        decode_timedelta=True,
    )
    for location_file, group in fcst_files.groupby("location_file")
]

# Concatenate across different location sets
fcst_raw = xr.concat(datasets, dim="location")
print(fcst_raw)

<xarray.Dataset> Size: 19MB
Dimensions:          (lead: 126, location: 92, ensemble: 200, nbnds: 2)
Coordinates:
    region           (location) object 736B 'pnw' 'pnw' ... 'japan' 'japan'
    color            (location) object 736B '#009988' '#009988' ... '#004488'
  * location         (location) object 736B 'Cypress' ... 'Lotte Arai'
    forecast_date    datetime64[ns] 8B 2025-12-01
  * ensemble         (ensemble) int64 2kB 0 1 2 3 4 5 ... 195 196 197 198 199
  * lead             (lead) timedelta64[ns] 1kB 1 days 2 days ... 126 days
    lat              (location) float64 736B dask.array<chunksize=(80,), meta=np.ndarray>
    lon              (location) float64 736B dask.array<chunksize=(80,), meta=np.ndarray>
    forecast_period  (lead, nbnds) datetime64[ns] 2kB 2025-12-01 ... 2026-04-05
Dimensions without coordinates: nbnds
Data variables:
    temp             (lead, location, ensemble) float32 9MB dask.array<chunksize=(126, 80, 200), meta=np.ndarray>
    precip           (lead, loc

#### Define add_season

This utility function will add `season` and `season_elapsed` coordinates to a `time`-denominated `Dataset`. This will be useful later as an input to `stack_by_season`.


In [ ]:
def add_season(
    ds: xr.Dataset,
    season_start_month: int,
    season_end_month: int,
) -> xr.Dataset:
    """Add season and season_elapsed coordinates to a dataset based on time.

    Args:
        ds: xarray Dataset with a 'time' coordinate
        season_start_month: Month when season starts (e.g. 10 for October)
        season_end_month: Month when season ends (e.g. 5 for May)

    Returns:
        xarray Dataset with new 'season' and 'season_elapsed' coordinates
    """
    time = ds.time
    month = time.dt.month.values
    year = time.dt.year.values

    # Calculate season_elapsed relative to each year's season start
    season_starts = pd.to_datetime([f"{y}-{season_start_month:02d}-01" for y in year])
    time_np = time.values.astype("datetime64[h]")
    season_starts_np = season_starts.values.astype("datetime64[h]")

    # For dates before October, use previous year's October 1
    # Account for leap years in the offset
    is_leap = pd.to_datetime(season_starts_np).is_leap_year
    year_hours = np.where(is_leap, 366 * 24, 365 * 24)

    hours = (
        (
            time_np
            - np.where(
                month < season_start_month,
                season_starts_np - np.timedelta64(1, "h") * year_hours,
                season_starts_np,
            )
        )
        .astype("timedelta64[h]")
        .astype(int)
    )
    is_summer = (month >= season_end_month) & (month < season_start_month)
    season_elapsed = np.where(is_summer, np.nan, hours)

    # Calculate season year
    season = year
    season = np.where(month < season_start_month, season - 1, season)
    season = np.where(is_summer, np.nan, season)

    # Add new coordinates
    ds = ds.assign_coords({"season": ("time", season), "season_elapsed": ("time", season_elapsed)})

    return ds

### Simplify the forecast

Now let's simplify the time series by removing information we don't need and reformatting it to be "day of season" aware.


In [ ]:
fcst = (
    fcst_raw.assign_coords(
        time=("lead", fcst_raw.forecast_date.values + fcst_raw.lead.values - pd.Timedelta(days=1)),
    )
    .swap_dims({"lead": "time"})
    .drop_vars(["lead", "forecast_period", "forecast_date"], errors="ignore")
)

# Trim data beyond the end of the ski season:
fcst = fcst.sel(time=slice(fcst_date, fcst_end))

# Add "day of season" coordinate so we can later use it as the primary coord
fcst = add_season(fcst, start_month, end_month)

print(fcst)

<xarray.Dataset> Size: 18MB
Dimensions:         (time: 121, location: 92, ensemble: 200)
Coordinates:
    region          (location) object 736B 'pnw' 'pnw' 'pnw' ... 'japan' 'japan'
    color           (location) object 736B '#009988' '#009988' ... '#004488'
  * location        (location) object 736B 'Cypress' 'Bachelor' ... 'Lotte Arai'
  * ensemble        (ensemble) int64 2kB 0 1 2 3 4 5 ... 194 195 196 197 198 199
    lat             (location) float64 736B dask.array<chunksize=(80,), meta=np.ndarray>
    lon             (location) float64 736B dask.array<chunksize=(80,), meta=np.ndarray>
  * time            (time) datetime64[ns] 968B 2025-12-01 ... 2026-03-31
    season          (time) float64 968B 2.025e+03 2.025e+03 ... 2.025e+03
    season_elapsed  (time) float64 968B 1.464e+03 1.488e+03 ... 4.344e+03
Data variables:
    temp            (time, location, ensemble) float32 9MB dask.array<chunksize=(121, 80, 200), meta=np.ndarray>
    precip          (time, location, ensemble) flo

## Get Historical Data

The `data_timeseries` function will load the historical daily ERA5 timeseries, which we can later compare to the forecast ensembles.


In [ ]:
# Get historical observed performance for each ski resort
hist_files = sk.data_timeseries(
    loc=loc,
    variable=vars,
    frequency=freq,
    field="vals",
    debias=debias,
    start=hist_start,
    end=min(fcst_end, today),
    force=force,
    verbose=verbose,
)

# Assemble each historical file into a single xarray dataset
hist = sk.load_multihistory(hist_files)
hist = add_season(hist, start_month, end_month)

print(hist)

<xarray.Dataset> Size: 9MB
Dimensions:         (time: 5903, location: 92)
Coordinates:
  * time            (time) datetime64[ns] 47kB 2009-10-01 ... 2025-11-28
  * location        (location) <U16 6kB '7 Springs' 'A-Basin' ... 'Zermatt'
    lat             (location) float64 736B 40.02 39.64 44.86 ... 39.89 46.02
    lon             (location) float64 736B -79.3 -105.9 -92.79 ... -105.8 7.752
    location_file   (location) object 736B 'resorts_1_00.csv' ... 'resorts_3_...
    season          (time) float64 47kB 2.009e+03 2.009e+03 ... 2.025e+03
    season_elapsed  (time) float64 47kB 0.0 24.0 48.0 ... 1.368e+03 1.392e+03
Data variables:
    temp            (time, location) float64 4MB 7.179 -3.984 ... -4.212 -7.503
    precip          (time, location) float64 4MB 0.3805 3.371 ... 0.002451


### Prepend history to forecast

We begin analyzing each ski season in October to account for snow accumulation before the mountains become skiable. If the forecast was generated after October, prepend the observed history to the forecast timeseries so that we account for weather before the forecast was generated.


In [ ]:
season_start = f"{year}-{start_month}-01"
if fcst.time[0] > np.datetime64(season_start):
    fcst = xr.concat(
        [
            hist.sel(time=slice(season_start, fcst.time[0] - pd.Timedelta(days=1)))
            .expand_dims(ensemble=fcst.ensemble)
            .transpose("ensemble", "time", "location"),
            fcst,
        ],
        dim="time",
    )
    print(fcst)

<xarray.Dataset> Size: 53MB
Dimensions:         (ensemble: 200, time: 180, location: 92)
Coordinates:
  * location        (location) object 736B '7 Springs' 'A-Basin' ... 'Zermatt'
  * ensemble        (ensemble) int64 2kB 0 1 2 3 4 5 ... 194 195 196 197 198 199
  * time            (time) datetime64[ns] 1kB 2025-10-01 ... 2026-03-31
    lat             (location) float64 736B 40.02 39.64 44.86 ... 39.89 46.02
    lon             (location) float64 736B -79.3 -105.9 -92.79 ... -105.8 7.752
    location_file   (location) object 736B 'resorts_1_00.csv' ... 'resorts_3_...
    season          (time) float64 1kB 2.025e+03 2.025e+03 ... 2.025e+03
    season_elapsed  (time) float64 1kB 0.0 24.0 48.0 ... 4.32e+03 4.344e+03
    region          (location) object 736B 'na_mid' 'rockies' ... 'europe'
    color           (location) object 736B '#777777' '#CC3311' ... '#33BBEE'
Data variables:
    temp            (ensemble, time, location) float64 26MB dask.array<chunksize=(200, 59, 30), meta=np.ndarr

## Elevation & Attributes

The snow model has an optional elevation parameter to increase accuracy. Let's use the Salient `geo` function to get the elevation at each location.


In [ ]:
elev_files = sk.geo(loc=loc, variables="elevation", force=force)
elev = xr.open_mfdataset(
    paths=elev_files["file_name"],
    concat_dim="location",
    combine="nested",
).load()
elev = elev.set_coords(elev.data_vars)
elev.attrs = {}  # prevent conflicts
elev = elev.drop_vars(["lat", "lon"]).reindex(location=fcst.location)

fcst = xr.merge([fcst, elev])
hist = xr.merge([hist, elev])

In [ ]:
# Make units more readable for plotting purposes instead of "mmm day-1"
fcst["precip"].attrs["units"] = hist["precip"].attrs["units"] = "mm/day"

In [ ]:
# Now that we're done preprocessing the data, let's un-chunk it
fcst = fcst.compute()
hist = hist.compute()

## Calculate Snow Water Equivalent

The `calc_swe` function builds on the `snow17` model to calculate the snow water equivalent (SWE) at each location and for each ensemble. It requires that the dataset input has data values `precip` and `temp`.


In [ ]:
def cached_swe(
    ds: xr.Dataset, src_files: pd.DataFrame, name: str, force: bool = False
) -> xr.Dataset:
    """Load or calculate snow water equivalent (SWE) with caching."""
    if force or "swe" not in ds:
        cache_path = os.path.join(
            sk.get_file_destination(),
            f"{name}_swe_{hashlib.md5(str(src_files).encode()).hexdigest()}.nc",
        )
        if os.path.exists(cache_path) and not force:
            ds["swe"] = xr.load_dataarray(cache_path)
        else:
            ds["swe"] = sk.hydro.calc_swe(ds, "time")
            ds["swe"].to_netcdf(path=cache_path, encoding={"location": {"dtype": str}})
    return ds


fcst = cached_swe(fcst, fcst_files, "fcst", force=force)

print(fcst.data_vars)

Data variables:
    temp     (ensemble, time, location) float64 26MB 15.87 6.352 ... -0.2939
    precip   (ensemble, time, location) float64 26MB 0.0 0.1836 ... 2.17
    swe      (ensemble, time, location) float64 26MB 0.0 0.0 0.0 ... 132.0 399.4


In [ ]:
hist = cached_swe(hist, hist_files, "hist", force=force)

print(hist.data_vars)

Data variables:
    temp     (time, location) float64 4MB 7.179 -3.984 9.07 ... -4.212 -7.503
    precip   (time, location) float64 4MB 0.3805 3.371 ... 0.00197 0.002451
    swe      (time, location) float64 4MB 0.0 3.371 0.0 ... 1.492 7.141 98.29


### Define `stack_by_season`

To calculate a seasonal average similar to the forecast's per-ensemble average we need to break the single linear `time` dimension into `season` + `season_elapsed` dimensions.


In [ ]:
def stack_by_season(ds):
    """Convert a dataset from time dimension to season/season_elapsed dimensions.

    Args:
        ds: xarray Dataset with time dimension and season/season_elapsed coordinates

    Returns:
        xarray Dataset with season and season_elapsed dimensions instead of time
    """
    valid_mask = ~ds["season"].isnull() & ~ds["season_elapsed"].isnull()
    ds_clean = ds.isel(time=valid_mask).copy()
    ds_clean["season"] = ds_clean.season.astype(int)
    ds_clean["season_elapsed"] = ds_clean.season_elapsed.astype(int)
    time_df = pd.DataFrame(
        {
            "season": ds_clean.season.values,
            "season_elapsed": ds_clean.season_elapsed.values,
            "time": ds_clean.time.values,
        }
    )

    ds_stacked = ds_clean.set_index(time=["season", "season_elapsed"])
    ds_final = ds_stacked.unstack("time")
    idx = pd.MultiIndex.from_product(
        [ds_final.season.values, ds_final.season_elapsed.values],
        names=["season", "season_elapsed"],
    )
    time_series = time_df.set_index(["season", "season_elapsed"])["time"].reindex(idx)
    ds_final["time"] = xr.DataArray(
        time_series.values.reshape(len(ds_final.season), len(ds_final.season_elapsed)),
        dims=["season", "season_elapsed"],
    )

    return ds_final

### Stack timeseries by season

So we can later merge, let's denominate the forecast and historical by `season_elapsed` instead of `time`. The `season` coordinate of `hist` represents each historical season, and the `ensemble` coordinate of `fcst` represents each potential future for this season.


In [ ]:
hist = stack_by_season(hist)
fcst = stack_by_season(fcst).squeeze("season")

print("Historical ------")
print(hist.data_vars)
print("Forecast ------")
print(fcst.data_vars)

Historical ------
Data variables:
    temp     (location, season, season_elapsed) float64 2MB 7.179 10.27 ... nan
    precip   (location, season, season_elapsed) float64 2MB 0.3805 4.513 ... nan
    swe      (location, season, season_elapsed) float64 2MB 0.0 0.0 ... nan nan
    time     (season, season_elapsed) datetime64[ns] 25kB 2009-10-01 ... NaT
Forecast ------
Data variables:
    temp     (ensemble, location, season_elapsed) float64 26MB 15.87 ... -0.2939
    precip   (ensemble, location, season_elapsed) float64 26MB 0.0 0.0 ... 2.17
    swe      (ensemble, location, season_elapsed) float64 26MB 0.0 0.0 ... 399.4
    time     (season_elapsed) datetime64[ns] 1kB 2025-10-01 ... 2026-03-31


## Calculate season start

Each location has a different historical start of its snow accumulation season. We'll later calculate historical averages only once the season has begun, because we only want to start the `temp` and `precip` anomaly clock once the snow accumulation season starts.


In [ ]:
# Find first hour that historically has meaningful snow accumulation
SNOW_THRESHOLD = 10  # mm
WINTER_SOLSTICE = 82 * 24  # Hours since October 1
hist_swe_avg = hist["swe"].mean("season")
season_start = (
    hist_swe_avg.where(hist_swe_avg > SNOW_THRESHOLD)
    .idxmin("season_elapsed")
    .clip(max=WINTER_SOLSTICE)
)
hist["season_start"] = fcst["season_start"] = season_start

# visualize -----
season_start_plt = (season_start / 24).to_pandas().sort_values()
ax = season_start_plt.plot(
    kind="bar",
    figsize=(15, 5),
    color=[fcst.sel(location=loc).color.item() for loc in season_start_plt.index],
)
plt.yticks([0, 31, (30 + 31), (30 + 31 + 31)], ["Oct", "Nov", "Dec", "Jan"])
plt.grid(True, axis="y", alpha=0.3)
ax.set_xticklabels(season_start_plt.index)
ax.set_ylabel(f"Days since {start_month}-01")
ax.set_xlabel("")
ax.set_title("Accumulation Start");

## Accumulate precipitation since season start


In [ ]:
for ds in [hist, fcst]:
    ds["precipc"] = (
        ds["precip"]
        .where(~(ds.season_elapsed < ds.season_start), 0)
        .cumsum(dim="season_elapsed", keep_attrs=True)
        .assign_attrs({"units": "mm", "long_name": "Cumulative precipitation"})
    )

print("Historical ------")
print(hist.data_vars)
print("Forecast ------")
print(fcst.data_vars)

Historical ------
Data variables:
    temp          (location, season, season_elapsed) float64 2MB 7.179 ... nan
    precip        (location, season, season_elapsed) float64 2MB 0.3805 ... nan
    swe           (location, season, season_elapsed) float64 2MB 0.0 0.0 ... nan
    time          (season, season_elapsed) datetime64[ns] 25kB 2009-10-01 ......
    season_start  (location) float64 736B 1.968e+03 552.0 ... 624.0 336.0
    precipc       (location, season, season_elapsed) float64 2MB 0.0 ... 161.1
Forecast ------
Data variables:
    temp          (ensemble, location, season_elapsed) float64 26MB 15.87 ......
    precip        (ensemble, location, season_elapsed) float64 26MB 0.0 ... 2.17
    swe           (ensemble, location, season_elapsed) float64 26MB 0.0 ... 3...
    time          (season_elapsed) datetime64[ns] 1kB 2025-10-01 ... 2026-03-31
    season_start  (location) float64 736B 1.968e+03 552.0 ... 624.0 336.0
    precipc       (ensemble, location, season_elapsed) float64 

## Calculate Anomalies

Combine the historical and forecast datasets into a single dataset so that we can make sure they are aligned by `location`.

We don't want to highlight resorts with below-average snowfall, so let's sort the dataset and cut out the bottom half.


In [ ]:
clim = hist.sel(season=slice(None, year - 1)).mean("season", keep_attrs=True)


def calc_anom(var, use_season_start=False, as_percent=False):
    """Calculate anomalies using resort-specific accumulation periods."""
    SPRING_EQUINOX = 170 * 24  # relative to October 1
    start_time = fcst.season_start if use_season_start else WINTER_SOLSTICE
    valid_fcst = (fcst.season_elapsed >= start_time) & (fcst.season_elapsed <= SPRING_EQUINOX)
    valid_clim = (clim.season_elapsed >= start_time) & (clim.season_elapsed <= SPRING_EQUINOX)
    with xr.set_options(keep_attrs=True):
        f = fcst[var].where(valid_fcst).mean("season_elapsed")
        c = clim[var].where(valid_clim).mean("season_elapsed")
        if as_percent:
            anom = 100 * (f / c)
            anom.attrs["units"] = "%"
        else:
            anom = f - c
    anom.attrs["long_name"] += " anomaly"
    return anom


anom = xr.Dataset(
    {
        "swe": calc_anom("swe", use_season_start=False, as_percent=True),
        "precip": calc_anom("precip", use_season_start=True, as_percent=True),
        "precipc": calc_anom("precipc", use_season_start=True, as_percent=True),
        "temp": calc_anom("temp", use_season_start=True, as_percent=False),
    }
)

# Focus on above-median resorts by snowfall
loc_avg = fcst.swe.mean(dim=["ensemble", "season_elapsed"], keep_attrs=True)
min_avg = loc_avg.median() + 10  # search just a bit above the median
anom = anom.sel(location=loc_avg[loc_avg > min_avg].location.values)

# Sort by SWE anomaly, highest first
anom = anom.sel(
    location=anom.swe.mean(dim=["ensemble"]).sortby(anom.swe.mean(dim=["ensemble"])).location
)

# Smooth out historical temperature for cleaner plotting
clim["temp"] = (
    clim["temp"]
    .rolling(season_elapsed=len(clim.season_elapsed) // 10, center=True, min_periods=1)
    .mean()
)

htime = pd.to_datetime(hist["time"].sel(season=year - 4).values) + pd.DateOffset(years=4)
clim = clim.assign(time=("season_elapsed", htime))
anom = anom.assign(time=("season_elapsed", htime))

# Visualize the per-resort averages so we can understand the analysis cutoff
loc_avg_plt = loc_avg.to_pandas().sort_values()
ax = loc_avg_plt.plot(
    kind="bar",
    figsize=(15, 5),
    color=[loc_avg.sel(location=locname).color.item() for locname in loc_avg_plt.index],
)
plt.axhline(y=min_avg, color="black", linestyle="--", alpha=0.5)
ax.set_xticklabels(loc_avg_plt.index)
ax.set_title("Historical mean snow accumulation")
ax.set_ylabel(f"Mean {loc_avg.attrs['long_name']} [{loc_avg.attrs['units']}]")
ax.set_xlabel("");

In [ ]:
# Denominate back in elapsed time for better plotting
fcst = fcst.swap_dims({"season_elapsed": "time"})
anom = anom.swap_dims({"season_elapsed": "time"})
clim = clim.swap_dims({"season_elapsed": "time"})

## Visualize Results


In [ ]:
def plot_boxes(
    fcst: xr.DataArray,
    title: str = "",
    vline=None,
    legend_loc: str = "center right",
    ax=None,
):
    """Plot predictions and observed values as a box-and-whisker plot."""
    # extract a table of seasonal averages (across all days) per location
    box_data = fcst.to_dataframe(dim_order=["location", "ensemble"])
    box_data = box_data[fcst.name].unstack(level=0).to_numpy()

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 10))  # Create a new figure if no axis is provided

    plt_box = ax.boxplot(
        box_data,
        showfliers=False,
        vert=False,
        tick_labels=fcst.location.values,
        patch_artist=True,
        meanline=True,
        showmeans=True,
        meanprops=dict(linewidth=1, color="white"),
        medianprops=dict(linewidth=0, color="gray", alpha=0),
        whiskerprops=dict(linewidth=0.7, color="gray"),
        capprops=dict(linewidth=0.7, color="gray"),
        boxprops=dict(linewidth=0.7, color="gray"),
    )

    [patch.set_facecolor(color) for patch, color in zip(plt_box["boxes"], fcst.color.values)]
    ax.set_xlabel(f"{fcst.long_name} ({fcst.units})")
    ax.set_title(title)
    legend_names = ["na_east", "ca_west", "japan", "europe", "pnw", "rockies", "na_west"]
    legend_handles = plt_box["boxes"][: len(legend_names)]

    if vline is not None:
        ax.axvline(vline, color="grey", linestyle="--", linewidth=1)

    if legend_loc == "none":
        ax.set_yticklabels([])
    else:
        leg = ax.legend(legend_handles, legend_names, loc=legend_loc)
        for patch, reg in zip(leg.get_patches(), legend_names):
            patch.set_facecolor(geo_colors[reg])


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 15))
plot_boxes(anom["swe"], vline=100, legend_loc="upper left", ax=ax1)
plot_boxes(anom["precip"], vline=100, legend_loc="none", ax=ax2)
plot_boxes(anom["temp"], vline=0, legend_loc="none", ax=ax3)
plt.tight_layout()

### Individual Location Detail


#### Define plotting functions


In [ ]:
def plot_ensembles(fcst, clim, var="temp", title=False, ylim=None):
    """Show ensemble values for a given variable."""
    x_val = "time"

    favg = fcst.mean(dim="ensemble", keep_attrs=True)
    clr = fcst["color"].values.tolist()

    ens_lines = fcst[var].plot.line(x=x_val, color=clr, alpha=0.03, add_legend=False)
    mean_line = favg[var].plot.line(x=x_val, color=clr, linewidth=2, add_legend=False)
    hist_line = clim[var].plot.line(x=x_val, color="black", linewidth=2, add_legend=False)

    winter_start = np.datetime64(f"{year}-12-21")
    winter_end = np.datetime64(f"{year+1}-03-20")
    plt.axvspan(winter_start, winter_end, color="grey", alpha=0.1)
    # Use provided ylim for text position, fall back to auto limits
    text_y = ylim[0] if ylim is not None else plt.ylim()[0]
    plt.text(
        winter_start + (winter_end - winter_start) / 2,
        text_y,
        "winter",
        color="grey",
        alpha=0.5,
        horizontalalignment="center",
        verticalalignment="bottom",
    )
    plt.title(fcst["location"].values if title else "")
    plt.xlabel("")
    if title:
        plt.legend(
            [ens_lines[0], mean_line[0], hist_line[0]],
            ["Forecast Ensembles", "Forecast Mean", "Historical Mean"],
            loc="upper left",
        )


def plot_locations(focus_names):
    """Plot swe/temp/precip timeseries for the named resort."""
    (fig, axs) = plt.subplots(
        nrows=len(focus_names),
        ncols=3,
        sharex=True,
        sharey="col",
        figsize=(5 * 3, 5 * len(focus_names)),
    )

    # Calculate y-axis limits based on forecast percentiles, omitting outliers
    fcst_focus = fcst.sel(location=focus_names)
    vars = ["swe", "precipc", "temp"]
    ylims = {var: fcst_focus[var].quantile([0.01, 0.99]).values for var in vars}

    for idx in range(len(focus_names)):
        fcst_loc = fcst.sel(location=focus_names[idx])
        clim_loc = clim.sel(location=focus_names[idx])
        plt.sca(axs[idx, 0])
        plot_ensembles(fcst_loc, clim_loc, "swe", title=True, ylim=ylims["swe"])
        plt.sca(axs[idx, 1])
        plot_ensembles(fcst_loc, clim_loc, "precipc", ylim=ylims["precipc"])
        plt.sca(axs[idx, 2])
        plot_ensembles(fcst_loc, clim_loc, "temp", ylim=ylims["temp"])
        plt.axhline(0, color="k", linestyle="--")

    # Set y-axis limits for each column based on percentiles
    for col_idx, var in enumerate(vars):
        axs[0, col_idx].set_ylim(ylims[var])

    # Show x-axis tick labels on all rows (not just the bottom)
    for ax in axs.flat:
        ax.tick_params(labelbottom=True)

#### Focus on key resorts


In [ ]:
plot_locations(["Lake Louise", "Killington", "Steamboat", "Niseko", "Chamonix"])